In [1]:
# Import required Packages

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time
from bs4 import BeautifulSoup
import pandas as pd

In [39]:
# Declare constants
training_source = 'Udemy'

# course id
aws_developer_associate = 0
aws_soultions_architect_associate = 1

# Course names
exam_names = ['']*2
exam_names[aws_developer_associate] ='aws-developer-associate'
exam_names[aws_soultions_architect_associate] ='aws-soultions-architect-associate' 

# Exam Codes
exam_codes = ['']*2
exam_codes[aws_developer_associate] ='dva-c01'
exam_codes[aws_soultions_architect_associate] ='saa-c02' 

# Course urls
urls =['']*2
urls[aws_developer_associate] = 'https://www.udemy.com/course/aws-certified-developer-associate-dva-c01/'
urls[aws_soultions_architect_associate] ='https://www.udemy.com/course/aws-certified-solutions-architect-associate-saa-c02/'

In [40]:
# Set Current Course & URL 
current_exam_scraped = aws_developer_associate
exam_code = exam_codes[current_exam_scraped]
exam_name = exam_names[current_exam_scraped]
start_url = urls[current_exam_scraped]
tech_name = 'aws'

file_name = training_source+'-'+exam_name

In [41]:
# Navigate to page with course contents and grab the page
with webdriver.Firefox() as driver:
    wait = WebDriverWait(driver, 10)
    driver.get(start_url)

    time.sleep(10)
 
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(3)


    # And grab the page HTML source
    html_page = driver.page_source
    
    # Save a local Copy
    with open(file_name+'.html','w') as file:
        file.write(html_page)

In [42]:
# Read the Saved HTML File 
source_page =''   
# Read the local saved copy
with open(file_name+'.html','r') as file:
    source_page = file.read() 

In [43]:
# Start Processing the file
# Parse the HTML with BeautifulSoup and create a soup object
soup = BeautifulSoup(source_page)

table_columns = []
table_columns.append('exam code')
table_columns.append('tech')
table_columns.append('training_source')
table_columns.append('exam_name')
table_columns.append('course_name')
table_columns.append('title_order')
table_columns.append('title')
table_columns.append('lectures')
table_columns.append('sub_section')
table_columns.append('duration')

table_data = []
title_order = 0

# Grap the course name 
course_name = soup.select('h1[class="clp-lead__title clp-lead__title--small"]')[0].text.strip()

# Select course list
course_content = soup.select('div[class*="section--panel--"]')  
for section in course_content:
    # Set loop Variables
    row_data = []
    title_order = title_order + 1
    no_of_lectures= ''
    
    # Extract Data
    ele = section.select('div[class*="section--section--"]')[0]
    title = ele.select('div[class*="section--section-title--"]')[0].text
    section_content = ele.select('div[class*="udlite-text-sm section--section-content--"]')[0]
    section_content = section_content.get_text(separator=" ", strip=True).split('•',1)[0]
    section_duration = ele.select('span')[0].text
    #sub_sections = section.select('div[class*="section--lecture-title-and-description--"]')
    
    section_panels = section.select('div[class*="section--panel-content--"]')
    # Set row data
    row_data.append(exam_code)
    row_data.append(tech_name)
    row_data.append(training_source)
    row_data.append(exam_name)
    row_data.append(course_name)
    row_data.append(title_order)
    row_data.append(title)
    row_data.append(no_of_lectures)
    row_data.append('')
    row_data.append(section_duration)
    table_data.append(row_data)
    
    sub_sections_count = 0
    for section_panel in section_panels:
        section_lectures = section_panel.select('div[class*="section--lecture-title-and-description--"]')
        for index,section_lecture in enumerate(section_lectures):
            sub_sections_count +=1
            sub_row = row_data.copy()
        
            sub_section = section_lecture
            sub_duration = section_panel.select('span[class*="section--lecture-content--"]')[index]
            sub_title = sub_section.select('span')[0]
        
            # set the rows values
            sub_row[8] = sub_title.text
            sub_row[9] = sub_duration.text
            table_data.append(sub_row)
    row_data[7] = sub_sections_count

In [44]:
# Create a Pandas DataFrame
df = pd.DataFrame(table_data, columns=table_columns)
df.to_excel(file_name+".xlsx") 
df

,exam code,tech,training_source,exam_name,course_name,title_order,title,lectures,sub_section,duration
0,dva-c01,aws,Udemy,aws-developer-associate,Ultimate AWS Certified Developer Associate 202...,1,Course Introduction - AWS Certified Developer ...,8,,14min
1,dva-c01,aws,Udemy,aws-developer-associate,Ultimate AWS Certified Developer Associate 202...,1,Course Introduction - AWS Certified Developer ...,,Course Introduction - AWS Certified Developer ...,02:18
2,dva-c01,aws,Udemy,aws-developer-associate,Ultimate AWS Certified Developer Associate 202...,1,Course Introduction - AWS Certified Developer ...,,PLEASE READ: Lectures you can skip if you took...,00:40
3,dva-c01,aws,Udemy,aws-developer-associate,Ultimate AWS Certified Developer Associate 202...,1,Course Introduction - AWS Certified Developer ...,,Create your AWS Account,01:53
4,dva-c01,aws,Udemy,aws-developer-associate,Ultimate AWS Certified Developer Associate 202...,1,Course Introduction - AWS Certified Developer ...,,AWS Account Activation Troubleshooting,02:05
...,...,...,...,...,...,...,...,...,...,...
398,dva-c01,aws,Udemy,aws-developer-associate,Ultimate AWS Certified Developer Associate 202...,29,Preparing for the Exam - AWS Certified Develop...,,Practice Test - AWS Certified Developer Associate,65 questions
399,dva-c01,aws,Udemy,aws-developer-associate,Ultimate AWS Certified Developer Associate 202...,30,Congratulations - AWS Certified Developer Asso...,3,,3min
400,dva-c01,aws,Udemy,aws-developer-associate,Ultimate AWS Certified Developer Associate 202...,30,Congratulations - AWS Certified Developer Asso...,,Congratulations - AWS Certified Developer Asso...,01:06
401,dva-c01,aws,Udemy,aws-developer-associate,Ultimate AWS Certified Developer Associate 202...,30,Congratulations - AWS Certified Developer Asso...,,THANK YOU!,01:32
